In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=01&fecha1_anomes=2020-10&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=01/10/2020&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [5]:
r = requests.get(url)

In [6]:
r.status_code

200

In [7]:
s = BeautifulSoup(r.text, 'lxml')

In [8]:
from selenium import webdriver

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path = './chromedriver.exe', options=options)

In [10]:
driver.get(url)

In [11]:
def get_segments(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class = "sc-hZSUBg gfeULV"]/div[@class = "sc-cLQEGU hyoued"]')
    schedule = {}
    escalas_tag = vuelo.find_elements_by_xpath('//div[@class="sc-cLQEGU dnKRNG"]//time')    
    escalas = list(map(lambda escala : escala.get_attribute('datetime') , escalas_tag))
    for i, segmento in enumerate(segmentos):
        aeropuertos_tag = segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//abbr[@class="sc-hrWEMg hlCkST"]')
        aeropuertos = list(map(lambda aeropuerto : aeropuerto.get_attribute('title') , aeropuertos_tag))
        horas_tag = segmento.find_elements_by_xpath('.//div[@class="sc-iujRgT jEtESl"]//time')
        horas = list(map(lambda hora : hora.get_attribute('datetime') , horas_tag))
        duracion = segmento.find_element_by_xpath('.//div[@class="sc-bMVAic hShZwU"]//time').get_attribute('datetime')
        modelo = segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]//span[@class="sc-gzOgki uTyOl"]').text
        vuelo = segmento.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]//b').text
        if segmento != segmentos[-1]:
            duracion_escala = escalas[i]
        else:
            duracion_escala = ''
        
        tramo = {'origen': aeropuertos[0],
                 'destino': aeropuertos[1],
                 'salida' : horas[0],
                 'llegada' : horas[1], 
                 'duracion' : duracion, 
                 'vuelo' : vuelo, 
                 'nave' : modelo, 
                 'duracion_escala' : duracion_escala}
      
        schedule[f'tramo {i+1}'] = tramo
       
    return schedule



In [12]:
def get_prices(vuelo):
    tarifas_tag = vuelo.find_elements_by_xpath('.//ul[@class="selection-tabs-fare-list"]//span[@class="price"]')
    monedas = list(map(lambda tarifa : tarifa.find_element_by_xpath('./span[@class="currency-symbol"]').text , tarifas_tag))
    valores = list(map(lambda tarifa : tarifa.find_element_by_xpath('./span[@class="value"]/span').text , tarifas_tag))
    tarifas = list(map(lambda moneda, valor : f'{moneda} {valor}', monedas, valores))
    categorias_tag = vuelo.find_elements_by_xpath('.//div[@class="selection-tabs"]//h4[@class="fare-category"]/span')
    categorias = list(map(lambda categoria : categoria.text , categorias_tag)) 
    tarifa_por_categoria = dict(zip(categorias, tarifas))
    return tarifa_por_categoria

In [13]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos. ')
    print('Iniciando scrapping...')
    info = []
    for vuelo in vuelos:
        vuelo.find_element_by_xpath('.//button[@class="sc-bdVaJa fuucJY"]').click()
        schedule = get_segments(vuelo)
        time.sleep(0.5)
        driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]/button').click()
        time.sleep(0.5)
        vuelo.click()
        time.sleep(0.5)
        prices = get_prices(vuelo)
        vuelo.click()
        info.append({'precios' : prices 
                    ,'schedule' : schedule
                    })
    return info

In [14]:
import time

In [15]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [16]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path = './chromedriver.exe', options=options)
driver.get(url)
delay = 15
try:
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La pagina termino de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    ('La pagina tardo demasiado en cargar')
    info_vuelos = []
    
driver.close()

info_vuelos


La pagina termino de cargar
Se encontraron 18 vuelos. 
Iniciando scrapping...


[{'precios': {},
  'schedule': {'tramo 1': {'origen': 'EZE',
    'destino': 'GRU',
    'salida': '18:00',
    'llegada': '20:50',
    'duracion': '2:50',
    'vuelo': 'LA8013',
    'nave': 'Airbus 320-200',
    'duracion_escala': '1:50'},
   'tramo 2': {'origen': 'GRU',
    'destino': 'MAD',
    'salida': '22:40',
    'llegada': '13:35',
    'duracion': '9:55',
    'vuelo': 'LA8064',
    'nave': 'Airbus 350-900',
    'duracion_escala': ''}}},
 {'precios': {},
  'schedule': {'tramo 1': {'origen': 'EZE',
    'destino': 'GRU',
    'salida': '15:35',
    'llegada': '18:25',
    'duracion': '2:50',
    'vuelo': 'LA8011',
    'nave': 'Airbus 320-200',
    'duracion_escala': '4:15'},
   'tramo 2': {'origen': 'GRU',
    'destino': 'MAD',
    'salida': '22:40',
    'llegada': '13:35',
    'duracion': '9:55',
    'vuelo': 'LA8064',
    'nave': 'Airbus 350-900',
    'duracion_escala': ''}}},
 {'precios': {},
  'schedule': {'tramo 1': {'origen': 'EZE',
    'destino': 'SCL',
    'salida': '11:50',


In [170]:
import pandas as pd

In [172]:
df = pd.DataFrame(data = info_vuelos)
df

,precios,schedule
0,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'GRU'..."
1,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'GRU'..."
2,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'SCL'..."
3,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'GRU'..."
4,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'SCL'..."
5,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'GRU'..."
6,"{'Light': 'ARS 51.069,80', 'Plus': 'ARS 56.173...","{'tramo 1': {'origen': 'EZE', 'destino': 'SCL'..."
7,"{'Light': 'ARS 62.399,45', 'Plus': 'ARS 67.502...","{'tramo 1': {'origen': 'EZE', 'destino': 'LIM'..."
8,"{'Light': 'ARS 58.214,30', 'Plus': 'ARS 63.521...","{'tramo 1': {'origen': 'EZE', 'destino': 'SCL'..."
9,"{'Light': 'ARS 58.214,30', 'Plus': 'ARS 63.521...","{'tramo 1': {'origen': 'EZE', 'destino': 'SCL'..."
